In [1]:
from pathlib import Path

from mndot_bid_etl.transform.abstract import read_abstract_csv
from mndot_bid_etl.transform.transform_bid import *

In [2]:
csv_file = Path("../data/csv/2022/220002.csv").resolve()
abstract = read_abstract_csv(csv_file)
bid_df = abstract.bid_df

In [3]:
bid_df.head()

,ContractId,SectionDescription,LineNumber,ItemNumber,ItemDescription,UnitPrice,Quantity,UnitName,Engineers (Unit Price),Engineers (Extended Amount),0000198793 (Unit Price),0000198793 (Extension),0000210000 (Unit Price),0000210000 (Extension),0000207897 (Unit Price),0000207897 (Extension)
0,220002,0001 - SP 0803-44,0005,2011601/01000,AS BUILT,15000,1,LS,15000,15000,"$9,800.00","$9,800.00","$12,000.00","$12,000.00","$9,800.00","$9,800.00"
1,220002,0001 - SP 0803-44,0010,2021501/00010,MOBILIZATION,250000,1,LS,250000,250000,"$209,000.00","$209,000.00","$317,000.00","$317,000.00","$216,000.00","$216,000.00"
2,220002,0001 - SP 0803-44,0015,2051501/00010,MAINT AND RESTORATION OF HAUL ROADS,1000,1,LS,1000,1000,$1.00,$1.00,$1.00,$1.00,$1.00,$1.00
3,220002,0001 - SP 0803-44,0020,2101505/00020,CLEARING,8000,1.4,ACRE,8000,11200,"$3,500.00","$4,900.00","$3,500.00","$4,900.00","$8,000.00","$11,200.00"
4,220002,0001 - SP 0803-44,0025,2101505/00030,GRUBBING,4000,1.4,ACRE,4000,5600,"$3,700.00","$5,180.00","$3,700.00","$5,180.00","$5,500.00","$7,700.00"


In [4]:
bid_df_filtered = filter_bid_columns(bid_df)
bid_df_filtered.head()

,ItemNumber,ItemDescription,Quantity,Engineers (Unit Price),0000198793 (Unit Price),0000210000 (Unit Price),0000207897 (Unit Price)
0,2011601/01000,AS BUILT,1,15000,"$9,800.00","$12,000.00","$9,800.00"
1,2021501/00010,MOBILIZATION,1,250000,"$209,000.00","$317,000.00","$216,000.00"
2,2051501/00010,MAINT AND RESTORATION OF HAUL ROADS,1,1000,$1.00,$1.00,$1.00
3,2101505/00020,CLEARING,1.4,8000,"$3,500.00","$3,500.00","$8,000.00"
4,2101505/00030,GRUBBING,1.4,4000,"$3,700.00","$3,700.00","$5,500.00"


In [5]:
bid_df_renamed = rename_bid_columns(bid_df_filtered)
bid_df_renamed.head()

,item_id,long_description,quantity,engineers,0000198793,0000210000,0000207897
0,2011601/01000,AS BUILT,1,15000,"$9,800.00","$12,000.00","$9,800.00"
1,2021501/00010,MOBILIZATION,1,250000,"$209,000.00","$317,000.00","$216,000.00"
2,2051501/00010,MAINT AND RESTORATION OF HAUL ROADS,1,1000,$1.00,$1.00,$1.00
3,2101505/00020,CLEARING,1.4,8000,"$3,500.00","$3,500.00","$8,000.00"
4,2101505/00030,GRUBBING,1.4,4000,"$3,700.00","$3,700.00","$5,500.00"


In [6]:
bid_df_formatted = format_bid_values(bid_df_renamed)
bid_df_formatted.head()

,item_id,long_description,quantity,engineers,0000198793,0000210000,0000207897
0,2011.601/01000,AS BUILT,1.0,1500000,980000,1200000,980000
1,2021.501/00010,MOBILIZATION,1.0,25000000,20900000,31700000,21600000
2,2051.501/00010,MAINT AND RESTORATION OF HAUL ROADS,1.0,100000,100,100,100
3,2101.505/00020,CLEARING,1.4,800000,350000,350000,800000
4,2101.505/00030,GRUBBING,1.4,400000,370000,370000,550000


In [7]:
bid_df_melted = melt_bid_df(bid_df_formatted)
bid_df_melted.head()

,item_id,long_description,quantity,bidder_id,unit_price
0,2011.601/01000,AS BUILT,1.0,engineers,1500000
1,2021.501/00010,MOBILIZATION,1.0,engineers,25000000
2,2051.501/00010,MAINT AND RESTORATION OF HAUL ROADS,1.0,engineers,100000
3,2101.505/00020,CLEARING,1.4,engineers,800000
4,2101.505/00030,GRUBBING,1.4,engineers,400000


In [8]:
bid_df_assigned = assign_contract_id(bid_df_melted, abstract.contract_id)
bid_df_assigned.head()

,item_id,long_description,quantity,bidder_id,unit_price,contract_id
0,2011.601/01000,AS BUILT,1.0,engineers,1500000,220002
1,2021.501/00010,MOBILIZATION,1.0,engineers,25000000,220002
2,2051.501/00010,MAINT AND RESTORATION OF HAUL ROADS,1.0,engineers,100000,220002
3,2101.505/00020,CLEARING,1.4,engineers,800000,220002
4,2101.505/00030,GRUBBING,1.4,engineers,400000,220002
